# path

In [26]:
source_directory = "/home/jacktran/RAG/experiment/files/"
target_directory = "/home/jacktran/RAG/experiment/target_files/"
csv_file = "/home/jacktran/RAG/experiment/processed/pdf_text.csv"

MODEL_ID = "panalexeu/xlm-roberta-ua-distilled"

# import

In [15]:
import os
import pandas as pd
import pymupdf

In [16]:
def extract_pdf(pdf_path, target_path):
    doc = pymupdf.open(pdf_path)
    with open(target_path, "wb") as out:
        for page in doc:
            text = page.get_text().encode("utf-8")
            out.write(text)
            out.write(bytes((12,)))

In [17]:
def extract_all_in_directory(source: str, target: str) -> None:
    if not os.path.exists(target):
        os.mkdir(target)

    for file in os.listdir(source):
        if file.endswith(".pdf"):
            file_path = os.path.join(source, file)
            target_file= file.split(".")[0] + ".txt"
            target_file_path = os.path.join(target, target_file)
            extract_pdf(file_path, target_file_path)
            print(f"extract from {file_path} to {target_file_path}")

In [8]:
extract_all_in_directory(source_directory, target_directory)

extract from /home/jacktran/RAG/experiment/files/927ac79f-30b6-420c-af2f-9229ba129bdf.pdf to /home/jacktran/RAG/experiment/target_files/927ac79f-30b6-420c-af2f-9229ba129bdf.txt
extract from /home/jacktran/RAG/experiment/files/SSI_Q3_25_a82298dac6.pdf to /home/jacktran/RAG/experiment/target_files/SSI_Q3_25_a82298dac6.txt
extract from /home/jacktran/RAG/experiment/files/Baocaocapnhat_VPB_Q3_VN.pdf to /home/jacktran/RAG/experiment/target_files/Baocaocapnhat_VPB_Q3_VN.txt
extract from /home/jacktran/RAG/experiment/files/5fa639a7-4e66-495c-9e6f-22732ac748f4.pdf to /home/jacktran/RAG/experiment/target_files/5fa639a7-4e66-495c-9e6f-22732ac748f4.txt
extract from /home/jacktran/RAG/experiment/files/download-with-token (1).pdf to /home/jacktran/RAG/experiment/target_files/download-with-token (1).txt
extract from /home/jacktran/RAG/experiment/files/3ac90064-2e23-43cc-9e7f-6b9ea653b552.pdf to /home/jacktran/RAG/experiment/target_files/3ac90064-2e23-43cc-9e7f-6b9ea653b552.txt
extract from /home/jac

# Cleaning

In [18]:
def cleaning(serie: pd.Series) -> pd.Series:
    return (
        serie
        .str.replace(r'\n{2,}', '. ', regex=True)
        .str.replace(r'[ \t\r\n]+', ' ', regex=True)
        .str.replace(' . ', '. ') 
        .str.strip()      
    )

In [19]:
df = pd.DataFrame(columns = ["file", "text"])
for file in os.listdir(target_directory):
    with open(target_directory + file, 'r', encoding= "UTF-8") as f:
        text = f.read()        
        row = {'file': file, 'text': text}
        df_tmp = pd.DataFrame([row])
        df_tmp["text"] = cleaning(df_tmp["text"])
        df = pd.concat([df, df_tmp], ignore_index = True)

In [20]:
df

,file,text
0,SSI_Q3_25_a82298dac6.txt,Giá mục tiêu 2026: % tăng giá: Cập nhật: TRIỂN...
1,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO CỘNG HÒA XÃ HÔ...
2,927ac79f-30b6-420c-af2f-9229ba129bdf.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NHƠN...
3,5641fb20-d894-499c-8b68-8477d24c992f.txt,1 BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NH...
4,VCB_cap-nhat_ACBS_10.txt,Cập nhật VCB – KHẢ QUAN 10/12/2025 Phòng Phân ...
5,download-with-token (1).txt,BÁO CÁO PHÂN TÍCH KĨ THUẬT ACB: MUA – 26.000 V...
6,3ac90064-2e23-43cc-9e7f-6b9ea653b552.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NHƠN...
7,Baocaocapnhat_VPB_Q3_VN.txt,Báo cáo cập nhật Ngân hàng TMCP Việt Nam Thịnh...


In [22]:
df.to_csv(csv_file)

# Chunking

In [28]:
from transformers import AutoTokenizer
MODEL_ID = "intfloat/multilingual-e5-large-instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

df = pd.read_csv(csv_file, index_col = 0)
df['n_tokens'] = df['text'].apply(lambda x: len(tokenizer.encode(x)))


Token indices sequence length is longer than the specified maximum sequence length for this model (2616 > 512). Running this sequence through the model will result in indexing errors


In [29]:
df

,file,text,n_tokens
0,SSI_Q3_25_a82298dac6.txt,Giá mục tiêu 2026: % tăng giá: Cập nhật: TRIỂN...,2616
1,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO CỘNG HÒA XÃ HÔ...,886
2,927ac79f-30b6-420c-af2f-9229ba129bdf.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NHƠN...,251
3,5641fb20-d894-499c-8b68-8477d24c992f.txt,1 BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NH...,1704
4,VCB_cap-nhat_ACBS_10.txt,Cập nhật VCB – KHẢ QUAN 10/12/2025 Phòng Phân ...,6556
5,download-with-token (1).txt,BÁO CÁO PHÂN TÍCH KĨ THUẬT ACB: MUA – 26.000 V...,987
6,3ac90064-2e23-43cc-9e7f-6b9ea653b552.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NHƠN...,1231
7,Baocaocapnhat_VPB_Q3_VN.txt,Báo cáo cập nhật Ngân hàng TMCP Việt Nam Thịnh...,6850


In [30]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def chunking_with_metadata(file_name, full_text, max_token=500):
    pages = full_text.split('\x0c')
    chunks_with_metadata = []
    current_chunk_sentences = []
    current_tokens = 0
    start_page = 1 

    for i, page_content in enumerate(pages):
        page_num = i + 1
        sentences = [s.strip() for s in page_content.split('. ') if s.strip()]
        
        for sent in sentences:
            sent_with_dot = sent + ". "
            token_ids = tokenizer.encode(sent_with_dot, add_special_tokens=False, truncation=True, max_length=1024)
            token_len = len(token_ids)
            
            if token_len > max_token:
                if current_chunk_sentences:
                    chunks_with_metadata.append({
                        'file': file_name,
                        'text': " ".join(current_chunk_sentences).strip(),
                        'page': f"{start_page}-{page_num}" if start_page != page_num else page_num
                    })
                    current_chunk_sentences = []
                    current_tokens = 0

                sub_chunks = [token_ids[i:i + max_token] for i in range(0, token_len, max_token)]
                for sub in sub_chunks:
                    chunks_with_metadata.append({
                        'file': file_name,
                        'text': tokenizer.decode(sub),
                        'page': page_num
                    })
                start_page = page_num
                continue

            if current_tokens + token_len > max_token:
                chunks_with_metadata.append({
                    'file': file_name,
                    'text': " ".join(current_chunk_sentences).strip(),
                    'page': f"{start_page}-{page_num}" if start_page != page_num else page_num
                })
                
                last_sentence = current_chunk_sentences[-1] if current_chunk_sentences else ""
                current_chunk_sentences = [last_sentence, sent] if last_sentence else [sent]
                current_tokens = len(tokenizer.encode(last_sentence + ". " + sent_with_dot, add_special_tokens=False, truncation=True))
                start_page = page_num
            else:
                current_chunk_sentences.append(sent)
                current_tokens += token_len

    if current_chunk_sentences:
        chunks_with_metadata.append({
            'file': file_name, 
            'text': " ".join(current_chunk_sentences).strip(),
            'page': f"{start_page}-{len(pages)}" if start_page != len(pages) else start_page
        })
        
    return chunks_with_metadata


all_records = []
for _, row in df.iterrows():
    if row['text']:
        res = chunking_with_metadata(row['file'], row['text'])
        all_records.extend(res)

df_vector = pd.DataFrame(all_records)

In [31]:
df_vector

,file,text,page
0,SSI_Q3_25_a82298dac6.txt,Giá mục tiêu 2026: % tăng giá: Cập nhật: TRIỂN...,1
1,SSI_Q3_25_a82298dac6.txt,Sau tăng vốn cổphiếu hiện đang giao dịch ởmức ...,1-2
2,SSI_Q3_25_a82298dac6.txt,"Lũy kế9 tháng đầu năm 2025, doanh thu của SSI ...",2
3,SSI_Q3_25_a82298dac6.txt,Danh mục tựdoanh tăng gần 5.000 tỷso với báo c...,2-4
4,SSI_Q3_25_a82298dac6.txt,P/B Giá dự phóng (VND) Tỷ trọng Giá dự phóng t...,4-5
5,SSI_Q3_25_a82298dac6.txt,Các chuyên viên phân tích nghiên cứu phụtrách ...,5
6,SSI_Q3_25_a82298dac6.txt,SSI CTCP Chứng khoán SSI Khuyến nghị Định nghĩ...,5
7,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO CỘNG HÒA XÃ HÔ...,1
8,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,"Đối chiếu với các quy định trên, thời hạn tối ...",1-2
9,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,Nơi nhận: KT HIỆU TRƯỞNG - Hiệu trưởng (bể ba...,2


In [32]:
df_vector['n_tokens'] = df_vector.text.apply(lambda x: len(tokenizer.encode(x)))
df_vector

,file,text,page,n_tokens
0,SSI_Q3_25_a82298dac6.txt,Giá mục tiêu 2026: % tăng giá: Cập nhật: TRIỂN...,1,489
1,SSI_Q3_25_a82298dac6.txt,Sau tăng vốn cổphiếu hiện đang giao dịch ởmức ...,1-2,492
2,SSI_Q3_25_a82298dac6.txt,"Lũy kế9 tháng đầu năm 2025, doanh thu của SSI ...",2,481
3,SSI_Q3_25_a82298dac6.txt,Danh mục tựdoanh tăng gần 5.000 tỷso với báo c...,2-4,467
4,SSI_Q3_25_a82298dac6.txt,P/B Giá dự phóng (VND) Tỷ trọng Giá dự phóng t...,4-5,497
5,SSI_Q3_25_a82298dac6.txt,Các chuyên viên phân tích nghiên cứu phụtrách ...,5,481
6,SSI_Q3_25_a82298dac6.txt,SSI CTCP Chứng khoán SSI Khuyến nghị Định nghĩ...,5,171
7,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,BỘ GIÁO DỤC VÀ ĐÀO TẠO CỘNG HÒA XÃ HÔ...,1,475
8,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,"Đối chiếu với các quy định trên, thời hạn tối ...",1-2,491
9,5fa639a7-4e66-495c-9e6f-22732ac748f4.txt,Nơi nhận: KT HIỆU TRƯỞNG - Hiệu trưởng (bể ba...,2,109


In [33]:
df_vector['n_tokens'].max()

np.int64(502)

# Embedding

In [34]:
from sentence_transformers import SentenceTransformer

In [35]:
model = SentenceTransformer(MODEL_ID)

Loading weights: 100%|██████████| 391/391 [00:00<00:00, 1119.44it/s, Materializing param=pooler.dense.weight]                               


In [36]:
def get_embedding(text: str):
    text = text.replace("\n", " ")
    return model.encode(text, normalize_embeddings=True).tolist()

df_vector["embeddings"] = df_vector["text"].apply(get_embedding)

In [37]:
import numpy as np

embeddings = np.array(df_vector["embeddings"].tolist())
embeddings.shape

(58, 1024)

# Test

In [39]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def query_top_n(user_query: str, df: pd.DataFrame, top_n: int = 5):
    instructional_query = f"query: {user_query}"
    
    query_vec = model.encode(instructional_query, normalize_embeddings=True).reshape(1, -1)
    
    doc_vecs = np.stack(df['embeddings'].values)
    
    scores = cosine_similarity(query_vec, doc_vecs)[0]
    
    top_indices = np.argsort(scores)[::-1][:top_n]
    
    results = []
    for idx in top_indices:
        results.append({
            "file_name": df.iloc[idx]['file'],
            "page": df.iloc[idx].get('page', 'N/A'),
            "text": df.iloc[idx]['text'],
            "score": round(float(scores[idx]), 4)
        })
    return results

results = query_top_n("Hội thảo Tham vấn về xây dựng các phương án phòng ngừa, ứng phó, bảo đảm an ninh môi trường biển của tỉnh Gia Lai được tổ chức vào thời gian và địa điểm cụ thể nào?", df_vector)

for res in results:
    print(f"[{res['score']}] File: {res['file_name']} (Trang: {res['page']})")
    print(f"Content: {res['text'][:200]}...")
    print("-" * 30)

[0.9124] File: 5641fb20-d894-499c-8b68-8477d24c992f.txt (Trang: 1)
Content: 1 BỘ GIÁO DỤC VÀ ĐÀO TẠO TRƯỜNG ĐẠI HỌC QUY NHƠN CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc Số: /KH-ĐHQN Gia Lai, ngày tháng 12 năm 2025 KẾ HOẠCH Tổ chức Hội thảo khoa học “Tham vấ...
------------------------------
[0.9099] File: 5641fb20-d894-499c-8b68-8477d24c992f.txt (Trang: 1-2)
Content: - Ban Giám đốc Công an tỉnh, lãnh đạo, chỉ huy PV01, PC03, PA02, PA03, PA04, chỉ huy 15 Công an xã, phường ven biển, Ban chỉ huy Bộ đội Biên phòng thuộc Bộ Chỉ huy Quân sự tỉnh - Đại diện lãnh đạo Việ...
------------------------------
[0.8738] File: 5641fb20-d894-499c-8b68-8477d24c992f.txt (Trang: 2-3)
Content: 4 Chương trình TT Thời gian Nội dung Người / đơn vị chủ trì thực hiện 1 13h30-14h00 Đón tiếp đại biểu, ổn định hội trường Trường Đại học Quy Nhơn + Công an tỉnh Gia Lai 2 14h00-14h10 Tuyên bố lý do, g...
------------------------------
[0.8445] File: 5641fb20-d894-499c-8b68-8477d24c992f.txt (Trang:

# Testing extraction

In [ ]:
import pymupdf
import pandas as pd
import re
import statistics

def is_uppercase_ratio(text, threshold=0.8):
    letters = [c for c in text if c.isalpha()]
    if not letters:
        return False
    upper = [c for c in letters if c.isupper()]
    return len(upper) / len(letters) >= threshold



def parse_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)

    lines = []

    for page_num, page in enumerate(doc, start=1):
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            if "lines" not in block:
                continue

            for line in block["lines"]:
                text = "".join([s["text"] for s in line["spans"]]).strip()
                if not text:
                    continue

                span = line["spans"][0]

                lines.append({
                    "page": page_num,
                    "text": text,
                    "font_size": span["size"],
                    "font": span["font"],
                    "flags": span["flags"]
                })

    body_font_size = statistics.mode([l["font_size"] for l in lines])

    results = []

    current_heading = None
    current_body = []
    current_page = None

    for line in lines:
        text = line["text"]
        is_bold = "Bold" in line["font"] or (line["flags"] & 2)

        is_heading_candidate = (
            is_bold
            or line["font_size"] > body_font_size
            or re.match(r"^\d+[\.\)]", text)
            or is_uppercase_ratio(text)
        )

        if is_heading_candidate:
            if current_heading and current_body:
                results.append({
                    "page": current_page,
                    "heading": current_heading,
                    "body": " ".join(current_body).strip()
                })

            current_heading = text
            current_body = []
            current_page = line["page"]

        else:
            if current_heading:
                current_body.append(text)

    if current_heading and current_body:
        results.append({
            "page": current_page,
            "heading": current_heading,
            "body": " ".join(current_body).strip()
        })

    return pd.DataFrame(results)

In [43]:
pdf_path =  "/home/jacktran/RAG/experiment/files/927ac79f-30b6-420c-af2f-9229ba129bdf.pdf"

df = parse_pdf(pdf_path)
df

,page,heading,body
0,1,"năm học 2025-2026, đại học chính quy",Căn cứ Thông báo số 4567/TB-ĐHQN ngày 26/12/20...
1,1,Nơi nhận:,- HT (để báo cáo); - Các Khoa (để thực hiện); ...
2,1,TS. Đinh Anh Tuấn,70 13 01


In [44]:
pdf_path =  "/home/jacktran/RAG/experiment/files/3ac90064-2e23-43cc-9e7f-6b9ea653b552.pdf"

df = parse_pdf(pdf_path)
df

,page,heading,body
0,1,"Ngôn ngữ Anh, Công tác xã hội, Kế toán, Quản l...",Trường Đại học Quy Nhơn thông báo tuyển sinh đ...
1,1,Chỉ tiêu,"1 Luật Xét kết quả học tập trung cấp, cao đẳng..."
2,1,"2. Đối tượng, điều kiện tuyển sinh",Đối tượng tuyển sinh là những người đã có bằng...
3,2,3. Thủ tục dự tuyển,Thí sinh nộp 01 bộ hồ sơ giấy (trực tiếp hoặc ...
4,2,Lai.,Hồ sơ gồm có: - Phiếu đăng ký xét tuyển; - Đối...
5,2,"4. Thời gian đào tạo, hình thức học và chuẩn đ...",Thời gian đào tạo được xác định dựa trên trình...
6,2,5. Phí tuyển sinh,Phí tuyển sinh: 400.000 đồng/thí sinh.
7,2,Thông tin liên hệ:,"- Phòng Đào tạo (bộ phận Vừa làm vừa học), Phò..."
8,3,3,Website: https://pdt.qnu.edu.vn - Phòng Kế hoạ...
9,3,Nơi nhận:,"- Hiệu trưởng (để báo cáo); - Phòng: KH-TC, CT..."
